In [2]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.naive_bayes import MultinomialNB
import mglearn
import nltk
from nltk.stem import PorterStemmer

%matplotlib inline
train = pd.read_csv("spooky-authors/train.zip", index_col=['id'])
test = pd.read_csv("spooky-authors/test.zip", index_col=['id'])
sample_submission = pd.read_csv("spooky-authors/sample_submission.zip", index_col=['id'])

Първо събирам train и test сетовете. По този начин ако има думи, които ги има само в теста CountVectorizer създаде колони за тях и в трейна. Когато тренирам модел ще ползвам само редовете от които идват от train сета.

In [3]:
combo = pd.concat([train, test])

In [4]:
print(train.shape)
print(test.shape)
print(combo.shape)

(19579, 2)
(8392, 1)
(27971, 2)


Идеята е да ползвам LDA, за да открия n на брой теми. С малко късмет различните автори ще са писали по различни теми и това ще помогне за тяхното идентифициране.

Използвам LDA с 20 теми (няма смисъл от повече, резултата не се променя особено).

In [5]:
from sklearn.decomposition import LatentDirichletAllocation
from scipy.sparse import coo_matrix, hstack

vectorizer = CountVectorizer(max_df=.15, max_features=10000)
X = vectorizer.fit_transform(combo.text)
lda = LatentDirichletAllocation(n_components=20, 
                                learning_method="batch", max_iter=15, random_state=0)
topics = lda.fit_transform(X)
topics.shape

(27971, 20)

Създавам си функции, които да извличат темите от техта и да го векторизират с TfidfVectorizer.

In [7]:
def vectorize_text(text):
    tfidf = TfidfVectorizer(ngram_range=(1, 2), min_df=2,
                                 max_df=0.8, lowercase=False)
    return tfidf.fit_transform(text)

In [8]:
def generate_topics(text, ld):
    vectorizer = CountVectorizer(max_df=.15, max_features=10000)
    vec = vectorizer.fit_transform(text)
    return ld.transform(vec)

Събирам двете sparse матрици в една.

In [9]:
def combine_topics_and_vectors(vectors, topics):
    sparse_topics = coo_matrix(topics)
    return hstack([vectors, sparse_topics])

Прилагам всичките операции чрез transform_text.

In [10]:
def transform_text(text, lda):
    vector = vectorize_text(text)
    topics = generate_topics(text, lda)
    return combine_topics_and_vectors(vector, topics)

Първо пробвам да предрека автора само с генерираните теми. 

In [11]:
model = MultinomialNB(alpha=0.01)
text = generate_topics(combo[:19579].text, lda)
print(cross_val_score(model, text, train.author, cv=3, n_jobs=3))
print(cross_val_score(model, text, train.author,cv=3, n_jobs=3, 
                      scoring='neg_log_loss'))

[ 0.40349265  0.40346307  0.4035249 ]
[-1.08185797 -1.08186615 -1.08166839]


Да видим какви теми намери LDA.

In [12]:
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vectorizer.get_feature_names())
mglearn.tools.print_topics(topics=range(15), feature_names=feature_names, sorting=sorting, topics_per_chunk=5, n_words=10)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
which         we            is            more          by            
or            our           this          than          years         
is            us            no            his           these         
from          were          at            no            were          
its           from          very          less          one           
some          which         be            which         many          
their         now           so            him           his           
are           or            not           now           this          
these         let           there         days          few           
natural       by            an            found         are           


topic 5       topic 6       topic 7       topic 8       topic 9       
--------      --------      --------      --------      --------      
his 

Прави ми впечатление, че повечето теми съдържат главно стоп думи, които са популярни за всеки текст. Може би е добра идея да се премахнат. 

Така и така ще ги махаме направо и един стемер да ударим.

In [13]:
stopwords = nltk.corpus.stopwords.words('english')
stemmer = PorterStemmer()
explore = combo.copy()
explore['no_stop'] = explore.text.apply(lambda s: " ".join([stemmer.stem(w) for w in str(s).split() if w.lower() not in stopwords]))

In [14]:
vectorizer_explore = CountVectorizer(max_df=.15, max_features=10000)
X_explore = vectorizer_explore.fit_transform(explore.no_stop)
lda_explore = LatentDirichletAllocation(n_components=20, 
                                learning_method="batch", max_iter=15, random_state=0)
topics_explore = lda_explore.fit_transform(X_explore)
topics_explore.shape

(27971, 20)

In [15]:
sorting = np.argsort(lda_explore.components_, axis=1)[:, ::-1]
feature_names = np.array(vectorizer_explore.get_feature_names())
mglearn.tools.print_topics(topics=range(15), feature_names=feature_names, sorting=sorting, topics_per_chunk=5, n_words=10)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
went          upon          old           night         one           
came          it            man           day           heard         
town          arm           thing         de            someth        
sound         lay           think         upon          could         
wind          head          like          sun           wall          
sea           west          mani          watch         hous          
one           still         ever          wander        old           
night         even          would         toward        ancient       
time          two           dream         rest          it            
sometim       saw           world         scene         told          


topic 5       topic 6       topic 7       topic 8       topic 9       
--------      --------      --------      --------      --------      
him 

Сега изглеждат по-добре. Има неща като murder, love.

In [25]:
model_explore = MultinomialNB(alpha=0.01)
text_explore = transform_text(explore[:19579].no_stop, lda_explore)
print(cross_val_score(model_explore, text_explore, train.author, cv=3, n_jobs=3))
print(cross_val_score(model_explore, text_explore, train.author,cv=3, n_jobs=3, 
                      scoring='neg_log_loss'))

[ 0.81541054  0.8210236   0.81333333]
[-0.4707332  -0.45912487 -0.46660947]


Няма подобрение, дори резултата се влоши.

Не се получи особено добре. Изглежда авторите не използват толкова различни теми (все пак всички са spooky :D)

Пробвам само с Tfidf векторизиран текст.

In [110]:
model = MultinomialNB(alpha=0.01)
text = vectorize_text(combo[:19579].text)
print(cross_val_score(model, text, train.author, cv=3, n_jobs=3))
print(cross_val_score(model, text, train.author,cv=3, n_jobs=3, 
                      scoring='neg_log_loss'))

[ 0.84987745  0.84400858  0.84061303]
[-0.39747551 -0.39623287 -0.4030639 ]


Това вече е по-добър резултат от този на лекции. Изглежда комбинирането на двата файла оказва влияние.

Да проверим все пак като се ползват и двете дали ще има полза. 

In [116]:
model = MultinomialNB(alpha=0.01)
text = transform_text(combo[:19579].text, lda)
print(cross_val_score(model, text, train.author, cv=3, n_jobs=3))
print(cross_val_score(model, text, train.author, cv=3, n_jobs=3, 
                      scoring='neg_log_loss'))


[ 0.8504902   0.84462151  0.8410728 ]
[-0.39707522 -0.39583773 -0.40274484]


Резултатите се подобриха съвсем минимално, но все пак се подобриха.

In [99]:
combo_vec = transform_text(combo.text, lda).tocsr()

In [100]:
model = MultinomialNB(alpha=0.01)
train_vec = combo_vec[:19579, :]
test_vec = combo_vec[19579:, :]
model.fit(train_vec, train.author)
test_predictions = model.predict_proba(test_vec)

In [101]:
submit_file = pd.DataFrame(test_predictions, columns=['EAP', 'HPL', 'MWS'], index=test.index)
submit_file.head(10)
submit_file.to_csv("~/Desktop/spooky_prediction.csv")

Резултата в kaggle e :
<img src="img/spooky_kaggle.png">

Резултата без LDA в kaggle беше малко по нисък - 0.369.... така, че LDA все пак помогна малко.

Също пробвах с LogisticRegression, SVC, RandomForest с много различни параметри, но винаги бяха значително по-слаби от Бейс, затова не ги включих в тетрадката.